In [1]:
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 343.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 457.2 kB/s eta 0:00:0000:0100:01


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 557.7 kB/s eta 0:00:0000:0100:01


# Load Environment Variables from .env file

We'll later pick connection strings from there. 

In [3]:
!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# e.g. "mongodb+srv://<username>:<password>@cluster0.mongodb.net/database.collection"
MONGO_INPUT_URI = os.getenv('MONGO_INPUT_URI', '')
MONGO_INPUT_URI != ''

True

In [6]:
from pyspark.sql import SparkSession

In [7]:
SparkSession.builder.master("local[*]").getOrCreate().stop()

In [8]:
SparkSession.builder.master("spark://spark-master:7077").getOrCreate().stop()

In [9]:
spark = SparkSession.builder \
    .appName('List Connectors') \
    .master('spark://spark-master:7077') \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:10.2.0') \
    .getOrCreate()

# Get a list of all loaded jars
jars = spark.sparkContext.getConf().get('spark.jars', 'none')
jars_list = jars.split(',')

# Print all loaded jars
for jar in jars_list:
    print(jar)

none


In [10]:
# Define Spark session
spark.stop()
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL MongoDB Atlas example") \
    .master('spark://spark-master:7077') \
    .config("spark.mongodb.input.uri", MONGO_INPUT_URI) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:10.2.0') \
    .getOrCreate()

print(spark.version)

# https://www.mongodb.com/docs/spark-connector/v10.2/scala/api/
df = spark.read.format('mongodb').load()

3.3.0


Py4JJavaError: An error occurred while calling o143.load.
: java.lang.ClassNotFoundException: 
Failed to find data source: mongodb. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:574)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:207)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:171)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Me

In [ ]:
# https://www.mongodb.com/docs/spark-connector/v10.2/scala/api/
df = spark.read.format('mongodb').load()

In [ ]:
pipeline = "[{ $match: { '_data.t': 'initial_info' } }, { $limit: 1 }]"

df = spark.read.format('mongodb') \
    .option('database', 'request_insights') \
    .option('collection', 'requests') \
    .option('pipeline', pipeline) \
    .load()